In [ ]:
! pip install --upgrade google-cloud-bigquery[bqstorage,pandas]
! pip install tensorflow-io-nightly
! pip install -U tensorflow
! pip install -U tensorboard
!pip install hyperdash
!mkdir tf_logs

     |████████████████████████████████| 163kB 4.8MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
  Found existing installation: google-cloud-bigquery 1.21.0
    Uninstalling google-cloud-bigquery-1.21.0:
      Successfully uninstalled google-cloud-bigquery-1.21.0


     |████████████████████████████████| 24.8MB 1.6MB/s 
     |████████████████████████████████| 86.3MB 59kB/s 
     |████████████████████████████████| 3.8MB 37.0MB/s 
     |████████████████████████████████| 450kB 38.8MB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.7.1 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Unins

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.0.0)
Requirement already up-to-date: tensorboard in /usr/local/lib/python3.6/dist-packages (2.0.2)
     |████████████████████████████████| 2.4MB 5.0MB/s 
  Found existing installation: grpcio 1.15.0
    Uninstalling grpcio-1.15.0:
      Successfully uninstalled grpcio-1.15.0


  Created wheel for hyperdash: filename=hyperdash-0.15.3-cp36-none-any.whl size=28553 sha256=5b24c1203509f76896c80b41c69a41e49c0312604f443b12676655c87e7f14ef
  Stored in directory: /root/.cache/pip/wheels/62/5f/af/bbcaeb6570e4904c14fb4c1b70fee559a3788182ce4d104ce7
Successfully built hyperdash


In [ ]:
%load_ext tensorboard

In [ ]:
PROJECT_ID = '' # @param {type:'string'}
BUCKET = '' # @param {type:'string'}

In [ ]:
import sys
import time

import tensorflow as tf
import tensorflow_io.arrow as arrow_io
from tensorflow_io.arrow.python.ops.arrow_dataset_ops import arrow_schema_to_tensor_types, arrow_to_tensor_type
import pyarrow

from google.cloud import bigquery
from google.cloud import bigquery_storage_v1beta1
from pyarrow import RecordBatchFileWriter, feather

if 'google.colab' in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()
else:
  import google.auth

from hyperdash import monitor_cell
!hyperdash signup --github

Opening browser, please wait. If something goes wrong, press CTRL+C to cancel.
 SSH'd into a remote machine, or just don't have access to a browser? Open this link in any browser and then copy/paste the provided access token: https://hyperdash.io/oauth/github/start?state=client_cli_manual 
Waiting for Github OAuth to complete.
If something goes wrong, press CTRL+C to cancel.
Access token: QsY/G1hdW41RnroSgrBfUMMKqcebG9/dA7k2KjnMZuI=
Successfully logged in! We also installed: wHicFRPl41ejE70y1GT29IC3ILXcarYCq7wBgiqK4J0= as your default API key


In [ ]:
credentials = None
if not 'google.colab' in sys.modules:
  credentials, your_project_id = google.auth.default(
      scopes=["https://www.googleapis.com/auth/cloud-platform"]
  )
bqclient = bigquery.Client(project=PROJECT_ID)
bqstorageclient = bigquery_storage_v1beta1.BigQueryStorageClient(credentials=credentials)

In [ ]:
%%time
query = """
SELECT 
  ifnull(trip_miles,0) trip_miles
  ,ifnull(pickup_community_area,0) pickup_community_area
  ,ifnull(dropoff_community_area,0) dropoff_community_area
  ,ifnull(fare,0) fare
  ,ifnull(tips,0) tips
  ,ifnull(tolls,0) tolls
  ,ifnull(extras,0) extras
  ,ifnull(trip_total,0) trip_total
  ,ifnull(pickup_latitude,0) pickup_latitude
  ,ifnull(pickup_longitude,0) pickup_longitude
  ,ifnull(dropoff_latitude, 0) dropoff_latitude
  ,ifnull(dropoff_longitude,0) dropoff_longitude
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE MOD(ABS(FARM_FINGERPRINT(unique_key)), 100) < 5
"""
start = time.time()
result = bqclient.query(query).result()
arrow_table = result.to_arrow(progress_bar_type='tqdm_notebook', bqstorage_client=bqstorageclient)
arrow_df = arrow_table.to_pandas()
end = time.time()
print(f'running time: {end-start}')


running time: 251.46856594085693
CPU times: user 15 s, sys: 2.97 s, total: 17.9 s
Wall time: 4min 11s


In [ ]:
%%time
df = arrow_table.to_pandas()

CPU times: user 288 ms, sys: 266 ms, total: 554 ms
Wall time: 146 ms


In [ ]:
%%time
query = """
SELECT 
  ifnull(trip_miles,0) trip_miles
  ,ifnull(pickup_community_area,0) pickup_community_area
  ,ifnull(dropoff_community_area,0) dropoff_community_area
  ,ifnull(fare,0) fare
  ,ifnull(tips,0) tips
  ,ifnull(tolls,0) tolls
  ,ifnull(extras,0) extras
  ,ifnull(trip_total,0) trip_total
  ,ifnull(pickup_latitude,0) pickup_latitude
  ,ifnull(pickup_longitude,0) pickup_longitude
  ,ifnull(dropoff_latitude, 0) dropoff_latitude
  ,ifnull(dropoff_longitude,0) dropoff_longitude
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE MOD(ABS(FARM_FINGERPRINT(unique_key)), 100) < 5
"""
start = time.time()
result = bqclient.query(query).result()
df = result.to_dataframe(progress_bar_type='tqdm_notebook', bqstorage_client=bqstorageclient)
end = time.time()
print(f'running time: {end-start}')


running time: 255.28152012825012
CPU times: user 35.4 s, sys: 4.43 s, total: 39.8 s
Wall time: 4min 15s


In [ ]:
%%time
%%bigquery --project $PROJECT_ID magic_command_df
SELECT 
  ifnull(trip_miles,0) trip_miles
  ,ifnull(pickup_community_area,0) pickup_community_area
  ,ifnull(dropoff_community_area,0) dropoff_community_area
  ,ifnull(fare,0) fare
  ,ifnull(tips,0) tips
  ,ifnull(tolls,0) tolls
  ,ifnull(extras,0) extras
  ,ifnull(trip_total,0) trip_total
  ,ifnull(pickup_latitude,0) pickup_latitude
  ,ifnull(pickup_longitude,0) pickup_longitude
  ,ifnull(dropoff_latitude, 0) dropoff_latitude
  ,ifnull(dropoff_longitude,0) dropoff_longitude
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE MOD(ABS(FARM_FINGERPRINT(unique_key)), 100) < 5

CPU times: user 5min 56s, sys: 17.6 s, total: 6min 14s
Wall time: 18min 35s


# ストレージに保存

### CSVに保存

In [ ]:
# @title CSVとして保存
# %%time
# df.to_csv('dataset.csv', index=False)
# !gsutil cp dataset.csv gs://${BUCKET}/arrow_test/dataset.csv

In [ ]:
# @title BatchRecordとして保存
# with pyarrow.OSFile('dataset.arrow', 'wb') as sink:
#   schema = arrow_table.schema
#   writer = RecordBatchFileWriter(sink, schema)
#   writer.write_table(arrow_table, 128)
#   writer.close()
#   !gsutil cp dataset.arrow gs://${BUCKET}/arrow_test/dataset.arrow

In [ ]:
# @title Featherとして保存
# feather.write_feather(arrow_df, dest='dataset.feather')
# !gsutil cp dataset.csv gs://${BUCKET}/arrow_test/dataset.feather

# 監視用のCallbackと訓練メソッドの準備

In [ ]:
from tensorflow.keras.callbacks import Callback
from hyperdash import Experiment

# https://qiita.com/shoji9x9/items/896204303a7a56321d4c よりお借りしました
class Hyperdash(Callback):
    def __init__(self, entries, exp):
        super(Hyperdash, self).__init__()
        self.entries = entries
        self.exp = exp

    def on_epoch_end(self, epoch, logs=None):
        for entry in self.entries:
            log = logs.get(entry)            
            if log is not None:
                self.exp.metric(entry, log)

In [ ]:
def train(experiment_name, tb_log_path='tf_logs', dataset=None):
  tf.keras.backend.clear_session()
  exp = Experiment(experiment_name)
  try:
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_log_path)
    hd_callback = Hyperdash(["mae", "mse"], exp)
  
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(6, input_shape=(11,), activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(3, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(1))
    optimizer = tf.keras.optimizers.Adam(amsgrad=True)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mse', 'mae'])
    # This builds the model for the first time:
    model.fit(dataset, epochs=5, callbacks=[tb_callback, hd_callback])
  finally:
    exp.end()

In [ ]:
columns = ['trip_miles','pickup_community_area',
                 'dropoff_community_area', 'fare','tips','tolls',
                 'extras','trip_total', 'pickup_latitude', 
                 'pickup_longitude', 'dropoff_latitude',
                 'dropoff_longitude']

# メモリからの訓練

## Arrow Table Dataset

In [ ]:
def preprocess(*data):
  feature = {c: element for c, element in zip(columns, data)}
  label = feature.pop('tips')
  feature = tf.stack([tf.cast(feature[c], tf.float64) for c in columns if c in feature], axis=1)
  
  return feature, label 

In [ ]:
%%monitor_cell create_arrow_dataset
arrow_dataset = arrow_io.ArrowDataset.from_record_batches(
    arrow_table.to_batches(128), batch_mode='auto', 
    output_types=tuple([type for type in arrow_schema_to_tensor_types(arrow_table.schema)[0]],),
    output_shapes=tuple([shape for shape in arrow_schema_to_tensor_types(arrow_table.schema)[1]])
    )
arrow_dataset = arrow_dataset.map(preprocess)

{'output_shapes': [TensorShape([None]), TensorShape([None]), TensorShape([None]), TensorShape([None]), TensorShape([None]), TensorShape([None]), TensorShape([None]), TensorShape([None]), TensorShape([None]), TensorShape([None]), TensorShape([None]), TensorShape([None])], 'output_types': [tf.float64, tf.int64, tf.int64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64]}
This run of create_arrow_dataset ran for 0:00:01 and logs are available locally at: /root/.hyperdash/logs/create-arrow-dataset/create-arrow-dataset_2019-12-03t22-47-56-160080.log


In [ ]:
%%monitor_cell "apache_arrow_dataset_loop_time"
for data in arrow_dataset:
  pass

This run of "apache_arrow_dataset_loop_time" ran for 0:00:30 and logs are available locally at: /root/.hyperdash/logs/apache-arrow-dataset-loop-time/apache-arrow-dataset-loop-time_2019-12-03t10-29-58-690346.log


In [ ]:
train('arrow_table_training', 'tf_log/arrow_table', arrow_dataset)

Epoch 1/5
  73592/Unknown - 377s 5ms/step - loss: 3.3354 - mse: 3.3354 - mae: 0.9865| mae:   0.986465 |
| mse:   3.335444 |
73592/73592 [==============================] - 377s 5ms/step - loss: 3.3354 - mse: 3.3354 - mae: 0.9865
Epoch 2/5
73572/73592 [============================>.] - ETA: 0s - loss: 3.0584 - mse: 3.0584 - mae: 0.9517| mae:   0.951640 |
| mse:   3.057893 |
73592/73592 [==============================] - 185s 3ms/step - loss: 3.0579 - mse: 3.0579 - mae: 0.9516
Epoch 3/5
73584/73592 [============================>.] - ETA: 0s - loss: 3.0014 - mse: 3.0014 - mae: 0.9436| mae:   0.943528 |
| mse:   3.001151 |
73592/73592 [==============================] - 184s 3ms/step - loss: 3.0012 - mse: 3.0012 - mae: 0.9435
Epoch 4/5
73579/73592 [============================>.] - ETA: 0s - loss: 2.8333 - mse: 2.8333 - mae: 0.9326| mae:   0.932492 |
| mse:   2.832904 |
73592/73592 [==============================] - 188s 3ms/step - loss: 2.8329 - mse: 2.8329 - mae: 0.9325
Epoch 5/5
73572/735

## Pandas DataFrame 

In [ ]:
columns_oriented_array = tuple([d for d in df.values.T])

In [ ]:
%%monitor_cell create_df_dataset
df_dataset = tf.data.Dataset.from_tensor_slices(columns_oriented_array)
df_dataset = df_dataset.batch(128).map(preprocess)

This run of create_df_dataset ran for 0:00:00 and logs are available locally at: /root/.hyperdash/logs/create-df-dataset/create-df-dataset_2019-12-03t22-48-54-705006.log


In [ ]:
train('pd_dataframe_training', 'tf_log/pandas_df', df_dataset)

Epoch 1/5
  73592/Unknown - 413s 6ms/step - loss: 2.5165 - mse: 2.5165 - mae: 0.9136| mae:   0.913622 |
| mse:   2.516550 |
73592/73592 [==============================] - 413s 6ms/step - loss: 2.5165 - mse: 2.5165 - mae: 0.9136
Epoch 2/5
73583/73592 [============================>.] - ETA: 0s - loss: 2.1537 - mse: 2.1537 - mae: 0.8804| mae:   0.880304 |
| mse:   2.153501 |
73592/73592 [==============================] - 219s 3ms/step - loss: 2.1535 - mse: 2.1535 - mae: 0.8803
Epoch 3/5
73581/73592 [============================>.] - ETA: 0s - loss: 2.1588 - mse: 2.1588 - mae: 0.8795| mae:   0.879382 |
| mse:   2.158573 |
73592/73592 [==============================] - 214s 3ms/step - loss: 2.1586 - mse: 2.1586 - mae: 0.8794
Epoch 4/5
73576/73592 [============================>.] - ETA: 0s - loss: 2.1227 - mse: 2.1227 - mae: 0.8822| mae:   0.882189 |
| mse:   2.122396 |
73592/73592 [==============================] - 217s 3ms/step - loss: 2.1224 - mse: 2.1224 - mae: 0.8822
Epoch 5/5
73591/735

In [ ]:
%%monitor_cell "df_dataset_loop_time"
for data in df_dataset:
  pass

This run of "df_dataset_loop_time" ran for 0:01:05 and logs are available locally at: /root/.hyperdash/logs/df-dataset-loop-time/df-dataset-loop-time_2019-12-03t11-10-36-295254.log


# 外部ストレージからの訓練

-urushi


In [ ]:
! gsutil cp gs://$BUCKET/arrow_test/dataset.csv dataset.csv 
! gsutil cp  gs://$BUCKET/arrow_test/dataset.feather dataset.feather

Copying gs://k-urushi/arrow_test/dataset.csv...
\ [1 files][719.4 MiB/719.4 MiB]                                                
Operation completed over 1 objects/719.4 MiB.                                    
Copying gs://k-urushi/arrow_test/dataset.feather...
/ [1 files][862.4 MiB/862.4 MiB]   91.6 MiB/s                                   
Operation completed over 1 objects/862.4 MiB.                                    


#CSVの訓練

In [ ]:
def csv_preprocess(features):
  labels = features.pop('tips')
  return tf.stack([tf.cast(d, tf.float64) for d in features.values()], axis=-1), labels

In [ ]:
%%monitor_cell create_csv_dataset
csv_dataset = tf.data.experimental.make_csv_dataset(
      'dataset.csv',
      batch_size=128, # Artificially small to make examples easier to show.
      num_epochs=1,
      column_names=columns,
      column_defaults = [0., 0, 0, 0., 0., 0., 0., 0., 0., 0., 0., 0.]).map(csv_preprocess)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.
This run of create_csv_dataset ran for 0:00:00 and logs are available locally at: /root/.hyperdash/logs/create-csv-dataset/create-csv-dataset_2019-12-03t22-49-00-352030.log


In [ ]:
%%monitor_cell loop_csv_dataset
for data in csv_dataset:
  pass

This run of loop_csv_dataset ran for 0:01:22 and logs are available locally at: /root/.hyperdash/logs/loop-csv-dataset/loop-csv-dataset_2019-12-03t22-49-25-239877.log


In [ ]:
train('csv_training', 'tf_log/csv', csv_dataset)


Epoch 1/5
  73592/Unknown - 346s 5ms/step - loss: 0.8220 - mse: 0.8220 - mae: 0.3706| mae:   0.370589 |
| mse:   0.822026 |
73592/73592 [==============================] - 346s 5ms/step - loss: 0.8220 - mse: 0.8220 - mae: 0.3706
Epoch 2/5
73589/73592 [============================>.] - ETA: 0s - loss: 0.6232 - mse: 0.6232 - mae: 0.3848| mae:   0.384839 |
| mse:   0.623189 |
73592/73592 [==============================] - 184s 3ms/step - loss: 0.6232 - mse: 0.6232 - mae: 0.3848
Epoch 3/5
73588/73592 [============================>.] - ETA: 0s - loss: 0.5825 - mse: 0.5825 - mae: 0.3943| mae:   0.394267 |
| mse:   0.582486 |
73592/73592 [==============================] - 184s 3ms/step - loss: 0.5825 - mse: 0.5825 - mae: 0.3943
Epoch 4/5
73588/73592 [============================>.] - ETA: 0s - loss: 0.5643 - mse: 0.5643 - mae: 0.3997| mae:   0.399727 |
| mse:   0.564330 |
73592/73592 [==============================] - 185s 3ms/step - loss: 0.5643 - mse: 0.5643 - mae: 0.3997
Epoch 5/5
73572/735

# featherからの読み込み

In [ ]:
columns = ['trip_miles','pickup_community_area',
           'dropoff_community_area', 'fare','tips','tolls',
           'extras','trip_total', 'pickup_latitude', 
           'pickup_longitude', 'dropoff_latitude',
           'dropoff_longitude']
dtype=tuple([tf.float64, tf.int64, tf.int64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64])
shape=tuple([[] for i in range(len(dtype))])

In [ ]:
%%monitor_cell create_feather_dataset
feather_dataset = arrow_io.ArrowFeatherDataset(
    ['dataset.feather'],
    columns=tuple([i for i in range(len(columns))]),
    output_types=dtype,
    output_shapes=shape,
    batch_size=128)
feather_dataset = feather_dataset.shuffle(buffer_size=10000).map(preprocess)

{'output_shapes': [TensorShape([None]), TensorShape([None]), TensorShape([None]), TensorShape([None]), TensorShape([None]), TensorShape([None]), TensorShape([None]), TensorShape([None]), TensorShape([None]), TensorShape([None]), TensorShape([None]), TensorShape([None])], 'output_types': [tf.float64, tf.int64, tf.int64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64]}
This run of create_feather_dataset ran for 0:00:00 and logs are available locally at: /root/.hyperdash/logs/create-feather-dataset/create-feather-dataset_2019-12-03t22-49-05-639368.log


In [ ]:
%%monitor_cell loop_feather_dataset
for data in feather_dataset:
  pass

This run of loop_feather_dataset ran for 0:00:17 and logs are available locally at: /root/.hyperdash/logs/loop-feather-dataset/loop-feather-dataset_2019-12-03t22-50-50-289882.log


In [ ]:
train('feather_training', 'tf_log/feather', feather_dataset)

Epoch 1/5
  73592/Unknown - 366s 5ms/step - loss: 2.6416 - mse: 2.6417 - mae: 0.9396| mae:   0.939557 |
| mse:   2.641657 |
73592/73592 [==============================] - 366s 5ms/step - loss: 2.6416 - mse: 2.6417 - mae: 0.9396
Epoch 2/5
73583/73592 [============================>.] - ETA: 0s - loss: 2.1429 - mse: 2.1430 - mae: 0.8905| mae:   0.890533 |
| mse:   2.142976 |
73592/73592 [==============================] - 180s 2ms/step - loss: 2.1429 - mse: 2.1430 - mae: 0.8905
Epoch 3/5
73579/73592 [============================>.] - ETA: 0s - loss: 2.0174 - mse: 2.0174 - mae: 0.8719| mae:   0.871841 |
| mse:   2.017188 |
73592/73592 [==============================] - 180s 2ms/step - loss: 2.0172 - mse: 2.0172 - mae: 0.8718
Epoch 4/5
73589/73592 [============================>.] - ETA: 0s - loss: 2.0010 - mse: 2.0011 - mae: 0.8717| mae:   0.871736 |
| mse:   2.001064 |
73592/73592 [==============================] - 179s 2ms/step - loss: 2.0010 - mse: 2.0011 - mae: 0.8717
Epoch 5/5
73572/735

In [ ]:
%tensorboard --logdir tf_logs